<a href="https://colab.research.google.com/github/varunsmn87/SolarcycleDTW/blob/main/DTW_Analog_Forecasting_Solar_Cycle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================
# DTW ANALOG SOLAR-CYCLE FORECAST: COLAB PIPELINE
# ============================================

import os, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

# fastdtw (Colab)
try:
    from fastdtw import fastdtw
except Exception:
    !pip -q install fastdtw
    from fastdtw import fastdtw

from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score
)
from scipy.stats import pearsonr

# ============================================
# 0. Repro & data-freeze settings
# ============================================

CUT_OFF = pd.Timestamp('2024-11-01')
RNG_SEED = 42
np.random.seed(RNG_SEED)

FROZEN_DIR = "data_frozen"
os.makedirs(FROZEN_DIR, exist_ok=True)
FROZEN_SILSO_CSV = os.path.join(
    FROZEN_DIR,
    f"SILSO_SN_ms_tot_V2_until_{CUT_OFF.date()}.csv"
)

STAMP_STR = f"Data ≤ {CUT_OFF.date()} | RNG={RNG_SEED}"

# ============================================
# Helpers
# ============================================

def stamp(ax):
    ax.text(
        0.99, 0.02, STAMP_STR,
        transform=ax.transAxes,
        ha='right', va='bottom', fontsize=8
    )

def save_fig(fig, basename):
    pdf_path = f"{basename}.pdf"
    eps_path = f"{basename}.eps"
    fig.savefig(pdf_path, bbox_inches='tight')
    fig.savefig(eps_path, format='eps', bbox_inches='tight')
    plt.close(fig)
    print(f"Saved {pdf_path} and {eps_path}")

def fill_ci(ax, x, lo, hi, label='98% CI', color='0.85'):
    ax.fill_between(x, lo, hi, color=color, label=label)

def true_rmse(y, yhat):
    try:
        return mean_squared_error(y, yhat, squared=False)
    except TypeError:
        return np.sqrt(mean_squared_error(y, yhat))

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.asarray(y_true, float), np.asarray(y_pred, float)
    mask = y_true != 0
    if not np.any(mask):
        return np.nan if np.any(y_pred != 0) else 0.0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.asarray(y_true, float), np.asarray(y_pred, float)
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2
    mask = denom != 0
    if not np.any(mask):
        return np.nan
    return np.mean(np.abs(y_true[mask] - y_pred[mask]) / denom[mask]) * 100


# ============================================
# 1. Load frozen SILSO data (monthly smoothed)
# ============================================

SILSO_URL = "https://www.sidc.be/SILSO/DATA/SN_ms_tot_V2.0.csv"

def _load_silso_raw(url=SILSO_URL):
    df_raw = pd.read_csv(url, header=None, delimiter=';')
    df_raw.columns = [
        'Year', 'Month', 'DateFraction', 'SmoothedSSN',
        'Definitive', 'Extra1', 'Extra2'
    ]
    df_raw['Date'] = pd.to_datetime(
        df_raw['Year'].astype(str) + '-' + df_raw['Month'].astype(str),
        errors='coerce'
    )
    df_raw = df_raw.dropna(subset=['Date']).set_index('Date')
    df_raw = df_raw[df_raw['SmoothedSSN'] >= 0]
    return df_raw

if os.path.exists(FROZEN_SILSO_CSV):
    df = pd.read_csv(FROZEN_SILSO_CSV, parse_dates=['Date']).set_index('Date')
else:
    raw = _load_silso_raw(SILSO_URL)
    df = raw.loc[:CUT_OFF].copy()
    df.reset_index().to_csv(FROZEN_SILSO_CSV, index=False)

print("Frozen SILSO range:", df.index.min().date(), "→", df.index.max().date(),
      "| rows:", len(df))


# ============================================
# 2. Cycle extraction (SC13–24) and basic figures
# ============================================

cycle_boundaries = {
    13: ('1889-03-01', '1901-02-01'),
    14: ('1901-03-01', '1913-07-01'),
    15: ('1913-08-01', '1923-07-01'),
    16: ('1923-08-01', '1933-02-01'),
    17: ('1933-03-01', '1944-01-01'),
    18: ('1944-02-01', '1954-03-01'),
    19: ('1954-04-01', '1964-09-01'),
    20: ('1964-10-01', '1976-02-01'),
    21: ('1976-03-01', '1986-08-01'),
    22: ('1986-09-01', '1996-07-01'),
    23: ('1996-08-01', '2008-12-01'),
    24: ('2009-01-01', '2019-12-01'),
}

def extract_cycle(cycle_num, df_source):
    start, end = pd.to_datetime(cycle_boundaries[cycle_num])
    segment = df_source.loc[start:end, 'SmoothedSSN'].values
    x = np.arange(len(segment))
    return x, segment

cycle_fits = {}
for sc in range(13, 25):
    try:
        cycle_fits[sc] = extract_cycle(sc, df)
    except Exception:
        pass

print("Cycles loaded:", sorted(cycle_fits.keys()))

# Full smoothed series (basic)
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)
ax.plot(df.index, df['SmoothedSSN'], color='black', linewidth=1)
for sc, (start, end) in cycle_boundaries.items():
    ax.axvline(pd.to_datetime(start), color='0.7', linestyle=':', linewidth=0.8)
    ax.text(pd.to_datetime(start), 0, f"SC{sc}",
            rotation=90, va='bottom', ha='right', fontsize=7)
ax.set_title("Monthly Smoothed Sunspot Number (SILSO, Frozen at Nov 2024)")
ax.set_xlabel("Year")
ax.set_ylabel("Smoothed SSN")
ax.grid(True, linestyle=':')
stamp(ax)
fig.tight_layout()
save_fig(fig, "fig_full_sunspot_series")


# ============================================
# 3. Convex reference + DTW analog blending + SC20–24 validation
# ============================================

def generate_convex_reference(length, peak_index=None):
    x = np.arange(length)
    if peak_index is None:
        return np.sin(np.pi * x / length)
    center = peak_index
    width = max(1, length // 3)
    d = np.abs(x - center) / width
    d = np.clip(d, 0, 1)
    return (1 - d) * np.sin(np.pi * x / length)

def dtw_curve_blend(target_curve, train_cycles, k=2):
    t = np.asarray(target_curve, float)
    t_len = len(t)
    t_min, t_max = float(np.min(t)), float(np.max(t))
    t_norm = (t - t_min) / (t_max - t_min + 1e-9)

    distances = []
    candidate_curves = {}

    for c in train_cycles:
        if c not in cycle_fits:
            continue
        _, y = cycle_fits[c]
        y = np.asarray(y, float)
        y_min, y_max = float(np.min(y)), float(np.max(y))
        y_norm = (y - y_min) / (y_max - y_min + 1e-9)
        y_adj = np.pad(y_norm, (0, max(0, t_len - len(y_norm))))[:t_len]
        candidate_curves[c] = y_adj
        dist, _ = fastdtw(t_norm, y_adj)
        distances.append((c, dist))

    if not distances:
        raise ValueError("No valid training cycles found.")

    distances.sort(key=lambda x: x[1])
    k_eff = min(k, len(distances))
    selected = distances[:k_eff]

    w = np.array([1.0 / (d + 1e-9) for (_, d) in selected], float)
    w /= w.sum()

    blended = np.zeros(t_len, float)
    for weight, (cid, _) in zip(w, selected):
        blended += weight * candidate_curves[cid]

    return blended, [c for (c, _) in selected]

def predict_cycle_length(parity, upto_cycle):
    past_lengths = []
    for c in range(13, upto_cycle):
        if c % 2 == parity and c in cycle_boundaries:
            start = pd.to_datetime(cycle_boundaries[c][0])
            end = pd.to_datetime(cycle_boundaries[c][1])
            months = (end.year - start.year) * 12 + (end.month - start.month) + 1
            past_lengths.append(months)
    if not past_lengths:
        return 132
    return int(round(np.mean(past_lengths)))

k_values = [2, 3, 4]
train_range = range(13, 20)
validation_results = {}

for k in k_values:
    rows = []
    for sc in range(20, 25):
        if sc not in cycle_fits:
            continue
        _, y_actual = cycle_fits[sc]
        dummy = generate_convex_reference(len(y_actual))
        train = [c for c in train_range if (c % 2 == sc % 2) and (c in cycle_fits)]
        blended, _ = dtw_curve_blend(dummy, train, k=k)
        amp = np.max(y_actual) - np.min(y_actual)
        base = np.min(y_actual)
        forecast = blended * amp + base
        mae = mean_absolute_error(y_actual, forecast)
        rmse = true_rmse(y_actual, forecast)
        peak_err = abs(np.max(y_actual) - np.max(forecast))
        timing_err = abs(int(np.argmax(y_actual) - np.argmax(forecast)))
        r2 = r2_score(y_actual, forecast)
        r = pearsonr(y_actual, forecast)[0] if (
            np.std(y_actual) > 0 and np.std(forecast) > 0
        ) else np.nan
        mape = mean_absolute_percentage_error(y_actual, forecast)
        smape = symmetric_mean_absolute_percentage_error(y_actual, forecast)
        rows.append((sc, mae, rmse, peak_err, timing_err, r2, r, mape, smape))
    validation_results[k] = rows

print("\n=== Validation Summary (SC20–24) ===")
print(f"{'k':<4}{'Avg MAE':>10}{'Avg RMSE':>10}{'Avg R2':>10}{'Avg r':>10}{'Avg MAPE':>12}{'Avg SMAPE':>12}")
print("-" * 68)
for k, res in validation_results.items():
    if not res:
        continue
    avg_mae = np.mean([t[1] for t in res])
    avg_rmse = np.mean([t[2] for t in res])
    avg_r2 = np.mean([t[5] for t in res])
    avg_r = np.nanmean([t[6] for t in res])
    avg_mape = np.nanmean([t[7] for t in res])
    avg_smape = np.nanmean([t[8] for t in res])
    print(f"{k:<4}{avg_mae:>10.2f}{avg_rmse:>10.2f}{avg_r2:>10.2f}{avg_r:>10.2f}{avg_mape:>12.2f}{avg_smape:>12.2f}")
print("-" * 68)

best_k = min(
    [k for k, res in validation_results.items() if res],
    key=lambda k: np.mean([t[2] for t in validation_results[k]])
)
print(f"\nBest k based on average RMSE: k={best_k}")

def forecast_for_cycle_with_k(sc, k, train_range):
    if sc not in cycle_fits:
        raise ValueError(f"Cycle SC{sc} not found in cycle_fits.")
    x_actual, y_actual = cycle_fits[sc]
    y_actual = np.asarray(y_actual, float)
    n = len(y_actual)
    ref = generate_convex_reference(n)
    train = [c for c in train_range if (c % 2 == sc % 2) and (c in cycle_fits)]
    if not train:
        raise ValueError(f"No training cycles found for SC{sc} with given train_range.")
    blended_norm, _ = dtw_curve_blend(ref, train, k=min(k, len(train)))
    blended_norm = np.asarray(blended_norm, float)
    if np.allclose(blended_norm, 0):
        y_forecast = np.zeros_like(y_actual)
    else:
        a, b = np.polyfit(blended_norm, y_actual, 1)
        y_forecast = a * blended_norm + b
    mae = mean_absolute_error(y_actual, y_forecast)
    rmse = true_rmse(y_actual, y_forecast)
    r2 = r2_score(y_actual, y_forecast)
    r = pearsonr(y_actual, y_forecast)[0] if (
        np.std(y_actual) > 0 and np.std(y_forecast) > 0
    ) else np.nan
    metrics = dict(MAE=mae, RMSE=rmse, R2=r2, r=r)
    return x_actual, y_actual, y_forecast, metrics

def plot_k_comparison_grid(sc, k_list=(1, 2, 3, 4), train_range=range(13, 20),
                           basename="fig_k_comparison_sc"):
    k_list = list(k_list)
    if len(k_list) != 4:
        raise ValueError("This plotting function is set up for exactly 4 k values.")

    fig, axes = plt.subplots(2, 2, figsize=(10, 8), sharex=True, sharey=True)
    axes = axes.flatten()

    y_min_global, y_max_global = None, None
    results = []
    for i, k in enumerate(k_list):
        x, y_act, y_pred, met = forecast_for_cycle_with_k(sc, k, train_range)
        results.append((k, x, y_act, y_pred, met))
        ymin, ymax = min(y_act.min(), y_pred.min()), max(y_act.max(), y_pred.max())
        y_min_global = ymin if y_min_global is None else min(y_min_global, ymin)
        y_max_global = ymax if y_max_global is None else max(y_max_global, ymax)

    for ax, (k, x, y_act, y_pred, met) in zip(axes, results):
        ax.plot(x, y_act, label=f"SC{sc} Actual", linewidth=1.5)
        ax.plot(x, y_pred, linestyle='--', linewidth=1.5,
                label=f"Forecast (k={k})")
        ax.set_title(f"SC{sc} — k = {k}")
        ax.grid(True, linestyle=':')
        text = (
            f"MAE={met['MAE']:.1f}\n"
            f"RMSE={met['RMSE']:.1f}\n"
            f"R²={met['R2']:.2f}\n"
            f"r={met['r']:.2f}" if not np.isnan(met['r']) else
            f"MAE={met['MAE']:.1f}\n"
            f"RMSE={met['RMSE']:.1f}\n"
            f"R²={met['R2']:.2f}\n"
            f"r=nan"
        )
        ax.text(
            0.98, 0.02, text,
            transform=ax.transAxes,
            ha='right', va='bottom', fontsize=7,
            bbox=dict(boxstyle='round,pad=0.3', fc='w')
        )

    for ax in axes[2:]:
        ax.set_xlabel("Months from cycle start")
    for ax in axes[::3]:
        ax.set_ylabel("Smoothed SSN")

    if y_min_global is not None and y_max_global is not None:
        margin = 0.05 * (y_max_global - y_min_global)
        for ax in axes:
            ax.set_ylim(y_min_global - margin, y_max_global + margin)

    fig.suptitle(f"Comparison of DTW Analog Forecasts for SC{sc} across k", y=0.995)
    fig.tight_layout(rect=[0, 0, 1, 0.97])

    fig.text(
        0.99, 0.01,
        STAMP_STR,
        ha='right',
        va='bottom',
        fontsize=8
    )

    outname = f"{basename}{sc}"
    save_fig(fig, outname)
    print(f"Saved k-comparison figure for SC{sc} as {outname}.pdf/.eps")

# Representative k-comparison
plot_k_comparison_grid(sc=24, k_list=(1, 2, 3, 4), train_range=range(13, 20),
                       basename="fig_k_comparison_sc")

# Validation grid (SC20–24, best k)
fig = plt.figure(figsize=(12, 10))
axes = fig.subplots(3, 2)
axes = axes.flatten()

idx = 0
for sc, mae, rmse, peak_err, timing_err, r2, r, mape, smape in validation_results[best_k]:
    x_actual, y_actual = cycle_fits[sc]
    dummy = generate_convex_reference(len(y_actual))
    train = [c for c in train_range if (c % 2 == sc % 2) and (c in cycle_fits)]
    blended, _ = dtw_curve_blend(dummy, train, k=best_k)
    amp = np.max(y_actual) - np.min(y_actual)
    base = np.min(y_actual)
    forecast = blended * amp + base

    ax = axes[idx]
    ax.plot(x_actual, y_actual, label=f"SC{sc} Actual", linewidth=1.5)
    ax.plot(x_actual, forecast, label=f"SC{sc} Forecast (k={best_k})",
            linestyle='--', linewidth=1.5)
    ax.set_title(f"SC{sc} (k={best_k})")
    ax.set_xlabel("Months")
    ax.set_ylabel("Smoothed SSN")
    ax.grid(True, linestyle=':')
    ax.legend(fontsize=8, framealpha=1.0)
    text = (
        f"MAE={mae:.1f}\nRMSE={rmse:.1f}\n"
        f"PeakErr={peak_err:.1f}\nPeakΔ={timing_err} mo\n"
        f"R²={r2:.2f}\nr={r:.2f}"
    )
    ax.text(0.98, 0.02, text, transform=ax.transAxes,
            ha='right', va='bottom', fontsize=7,
            bbox=dict(boxstyle='round,pad=0.3', fc='wheat'))
    idx += 1

for j in range(idx, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle("Validation: SC20–24 (DTW Analog Forecast, convex reference)", y=0.995)
fig.tight_layout(rect=[0, 0, 1, 0.97])
save_fig(fig, "fig_sc20_24_validation_grid")


# ============================================
# 4. SC26 DTW distances & analog justification
# ============================================

even_pool_all = [c for c in range(14, 25) if (c % 2 == 0) and (c in cycle_fits)]

SC26_LENGTH = 129
SC26_PEAK = 151.66
SC26_PEAK_MONTH = 42

ref26 = generate_convex_reference(SC26_LENGTH, peak_index=SC26_PEAK_MONTH)

distances_even = []
for c in even_pool_all:
    _, y = cycle_fits[c]
    y = np.asarray(y, float)
    y_min, y_max = np.min(y), np.max(y)
    y_norm = (y - y_min) / (y_max - y_min + 1e-9)
    y_adj = np.pad(y_norm, (0, max(0, len(ref26) - len(y_norm))))[:len(ref26)]
    t_min, t_max = np.min(ref26), np.max(ref26)
    t_norm = (ref26 - t_min) / (t_max - t_min + 1e-9)
    d, _ = fastdtw(t_norm, y_adj)
    distances_even.append((c, d))

distances_even.sort(key=lambda x: x[1])

print("\nDTW distances to SC26 reference (even cycles):")
for c, d in distances_even:
    print(f"  SC{c}: {d:.2f}")

fig = plt.figure(figsize=(8, 5))
ax = fig.add_subplot(111)
labels = [f"SC{c}" for (c, _) in distances_even]
vals = [d for (_, d) in distances_even]
bars = ax.bar(labels, vals, color='0.8')
top2 = [c for (c, _) in distances_even[:2]]
for bar, (c, _) in zip(bars, distances_even):
    if c in top2:
        bar.set_color('black')
ax.set_ylabel("DTW Distance")
ax.set_title("DTW Distance of Even Cycles to SC26 Reference Curve")
ax.grid(True, axis='y', linestyle=':')
fig.tight_layout()
save_fig(fig, "fig_sc26_dtw_distances")

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
x_ref = np.arange(len(ref26))
ax.plot(x_ref, ref26 / np.max(ref26), label="SC26 Reference (normalized)",
        linewidth=2.0, color='black')
for c, _ in distances_even:
    _, y = cycle_fits[c]
    y = np.asarray(y, float)
    yn = (y - y.min()) / (y.max() - y.min() + 1e-9)
    xr = np.linspace(0, len(ref26) - 1, len(yn))
    ax.plot(xr, yn, linewidth=1.0, linestyle=':',
            label=f"SC{c}" if c in top2 else None,
            color='red' if c in top2 else '0.7')
ax.set_title("Even Cycles vs SC26 Reference (Normalized, Highlighting Selected Analogs)")
ax.set_xlabel("Relative Months")
ax.set_ylabel("Normalized SSN")
ax.grid(True, linestyle=':')
handles, labels_ = ax.get_legend_handles_labels()
ax.legend(handles, labels_, fontsize=8, framealpha=1.0)
fig.tight_layout()
save_fig(fig, "fig_sc26_even_cycles_overlay")


# ============================================
# 5. Phase-binned residuals (odd/even)
# ============================================

BINS = 12

def phase_bin_indices(n, bins=BINS):
    phases = np.linspace(0.0, 1.0, n, endpoint=True)
    idx = (phases * bins).astype(int)
    idx = np.minimum(idx, bins - 1)
    return idx

def loo_pred_for_cycle(cyc, pool, k=2):
    if cyc not in cycle_fits:
        return None
    x_act, y_act = cycle_fits[cyc]
    n = len(y_act)
    ref = generate_convex_reference(n)
    train = [c for c in pool if c != cyc and c in cycle_fits]
    if not train:
        return None
    y_raw, _ = dtw_curve_blend(ref, train, k=min(k, len(train)))
    y_raw = np.asarray(y_raw, float)
    y_act = np.asarray(y_act, float)
    if np.allclose(y_raw, 0):
        return np.zeros_like(y_act)
    a, b = np.polyfit(y_raw, y_act, 1)
    return a * y_raw + b

def build_phase_residual_bins(pool_cycles, label=""):
    bins_raw = [[] for _ in range(BINS)]

    for cyc in pool_cycles:
        y_pred = loo_pred_for_cycle(cyc, pool_cycles, k=2)
        if y_pred is None:
            continue
        _, y_act = cycle_fits[cyc]
        y_act = np.asarray(y_act, float)
        n = min(len(y_act), len(y_pred))
        r = y_act[:n] - y_pred[:n]
        idx = phase_bin_indices(n, bins=BINS)
        for t in range(n):
            bins_raw[idx[t]].append(float(r[t]))

    bins_zero = []
    print(f"\nResidual binning stats ({label}):")
    print(f"{'Bin':>3}{'N':>6}{'Mean(raw)':>12}{'Std(raw)':>12}{'Mean(zero)':>14}{'Std(zero)':>12}")
    print("-" * 65)
    for b in range(BINS):
        arr = np.asarray(bins_raw[b], float)
        if arr.size == 0:
            bins_zero.append([])
            print(f"{b:3d}{0:6d}{0.0:12.2f}{0.0:12.2f}{0.0:14.2f}{0.0:12.2f}")
            continue
        mean_raw = float(np.mean(arr))
        std_raw = float(np.std(arr))
        arr_zero = arr - mean_raw
        mean_zero = float(np.mean(arr_zero))
        std_zero = float(np.std(arr_zero))
        bins_zero.append(arr_zero.tolist())
        print(f"{b:3d}{arr.size:6d}{mean_raw:12.2f}{std_raw:12.2f}{mean_zero:14.2f}{std_zero:12.2f}")
    print("-" * 65)
    print(f"{label}: total residual samples = {sum(len(b) for b in bins_raw)}")
    return bins_raw, bins_zero

odd_pool_all = [c for c in range(13, 25) if (c % 2 == 1) and (c in cycle_fits)]
even_pool_all_for_resid = [c for c in range(13, 25) if (c % 2 == 0) and (c in cycle_fits)]

odd_bins_raw, odd_bins_zero = build_phase_residual_bins(odd_pool_all, label="Odd cycles (SC13–23)")
even_bins_raw, even_bins_zero = build_phase_residual_bins(even_pool_all_for_resid, label="Even cycles (SC14–24)")

fig = plt.figure(figsize=(8, 3))
ax = fig.add_subplot(111)
x = np.linspace(0, 1, 200)
y = np.sin(np.pi * x)
ax.plot(x, y, color='black', linewidth=1.5)
for b in range(BINS + 1):
    xb = b / BINS
    ax.axvline(xb, color='0.7', linestyle=':')
ax.set_xlabel("Normalized Cycle Phase")
ax.set_ylabel("Normalized SSN (schematic)")
ax.set_title("Phase-Binning Schematic (12 Equal Phase Bins)")
ax.text(0.01, 0.9, "Rise", transform=ax.transAxes, fontsize=9)
ax.text(0.45, 0.9, "Peak", transform=ax.transAxes, fontsize=9, ha='center')
ax.text(0.95, 0.9, "Decay", transform=ax.transAxes, fontsize=9, ha='right')
ax.grid(True, linestyle=':')
fig.tight_layout()
save_fig(fig, "fig_residual_phase_binning_schematic")

fig = plt.figure(figsize=(15, 8))
axes = fig.subplots(3, 4)
axes = axes.flatten()
for b in range(BINS):
    ax = axes[b]
    arr = np.asarray(even_bins_zero[b], float)
    if arr.size > 0:
        ax.hist(arr, bins=20, edgecolor='black')
    ax.set_title(f"Bin {b+1}")
    ax.set_xlim(-80, 80)
fig.suptitle("Even Cycles: Zero-Mean Residuals per Phase Bin", y=0.995)
fig.tight_layout(rect=[0, 0, 1, 0.97])
save_fig(fig, "fig_even_residual_histograms_zero_mean")

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(111)
bins_idx = np.arange(1, BINS + 1)
odd_std = [np.std(b) if len(b) > 0 else 0.0 for b in odd_bins_zero]
even_std = [np.std(b) if len(b) > 0 else 0.0 for b in even_bins_zero]
ax.plot(bins_idx, odd_std, '-o', label='Odd cycles', linewidth=1.5)
ax.plot(bins_idx, even_std, '-s', label='Even cycles', linewidth=1.5)
ax.set_xlabel("Phase Bin")
ax.set_ylabel("Std of Zero-Mean Residuals")
ax.set_title("Residual Dispersion vs Phase Bin (Odd vs Even)")
ax.grid(True, linestyle=':')
ax.legend(framealpha=1.0)
fig.tight_layout()
save_fig(fig, "fig_residual_std_vs_phase")

print("\nResidual binning: zero-mean residuals per phase bin used for bootstrap CI.")


# ============================================
# 6. Phase-aware residual bootstrap CI
# ============================================

def bootstrap_ci_phase_aligned(forecast, bins_zero, draws=5000, q=(0.01, 0.99), seed=RNG_SEED):
    rng = np.random.default_rng(seed)
    f = np.asarray(forecast, float)
    n = len(f)
    idx = phase_bin_indices(n, bins=len(bins_zero))
    lo = np.empty(n, float)
    hi = np.empty(n, float)
    for t in range(n):
        pool = np.asarray(bins_zero[idx[t]], float)
        if pool.size == 0:
            lo[t] = f[t]
            hi[t] = f[t]
        else:
            samples = rng.choice(pool, size=draws, replace=True) + f[t]
            lo[t] = np.quantile(samples, q[0])
            hi[t] = np.quantile(samples, q[1])
    return lo, hi


# ============================================
# 7. SC25 & SC26 forecasts + CIs + stitched plot
# ============================================

# SC25 forecast (offline, odd parity, k=2)
train_odd = [c for c in range(13, 25) if (c % 2 == 1) and (c in cycle_fits)]
sc25_length = predict_cycle_length(1, 25)
ref25 = generate_convex_reference(sc25_length)
forecast25_raw, analogs25 = dtw_curve_blend(ref25, train_odd, k=2)

amps25 = [np.max(cycle_fits[c][1]) - np.min(cycle_fits[c][1]) for c in train_odd]
base25 = [np.min(cycle_fits[c][1]) for c in train_odd]
forecast25 = forecast25_raw * np.mean(amps25) + np.mean(base25)

print("\nSC25 forecast (offline):")
print("  Analogs:", analogs25)
print("  Length:", len(forecast25), "months")
print(f"  Peak: {np.max(forecast25):.2f} at month {np.argmax(forecast25)}")

lo25, hi25 = bootstrap_ci_phase_aligned(forecast25, odd_bins_zero, draws=5000, q=(0.01, 0.99))

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
x25 = np.arange(len(forecast25))
fill_ci(ax, x25, lo25, hi25, label='98% CI')
ax.plot(x25, forecast25, label='SC25 Forecast', color='black', linewidth=1.8)
ax.set_title("Forecasted Solar Cycle 25 (offline) with 98% Phase-Bootstrapped CI")
ax.set_xlabel("Months from SC25 Start (model index)")
ax.set_ylabel("Smoothed SSN")
ax.grid(True, linestyle=':')
ax.legend(framealpha=1.0)
stamp(ax)
fig.tight_layout()
save_fig(fig, "fig_sc25_forecast")

# SC26 forecast (even parity, enforced analogs via k=2)
ref26 = generate_convex_reference(SC26_LENGTH, peak_index=SC26_PEAK_MONTH)
forecast26_raw, analogs26 = dtw_curve_blend(ref26, even_pool_all, k=2)
max_raw = np.max(forecast26_raw) if np.max(forecast26_raw) != 0 else 1.0
forecast26 = (forecast26_raw / max_raw) * SC26_PEAK

print("\nSC26 forecast:")
print("  Analogs:", analogs26)
print("  Length:", len(forecast26), "months")
print(f"  Peak: {np.max(forecast26):.2f} at month {np.argmax(forecast26)}")

lo26, hi26 = bootstrap_ci_phase_aligned(forecast26, even_bins_zero, draws=5000, q=(0.01, 0.99))

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
x26 = np.arange(len(forecast26))
fill_ci(ax, x26, lo26, hi26, label='98% CI', color='0.90')
ax.plot(x26, forecast26, label='SC26 Forecast', color='black', linewidth=1.8)
for c in analogs26:
    if c in cycle_fits:
        x, y = cycle_fits[c]
        ax.plot(x[:SC26_LENGTH], y[:SC26_LENGTH], label=f'Analog SC{c}', linewidth=1.0)
ax.set_title("Forecasted Solar Cycle 26 (k=2) with 98% CI")
ax.set_xlabel("Months from SC26 Start (model index)")
ax.set_ylabel("Smoothed SSN")
ax.grid(True, linestyle=':')
ax.legend(fontsize=8, framealpha=1.0)
stamp(ax)
fig.tight_layout()
save_fig(fig, "fig_sc26_forecast")

# Stitch SC25 + SC26
blend_window = 6
bw = min(blend_window, len(forecast25), len(forecast26))
w_cos = np.cos(np.linspace(0, np.pi, bw)) ** 2

sc25_main, sc25_tail = forecast25[:-bw], forecast25[-bw:]
sc26_head, sc26_main = forecast26[:bw], forecast26[bw:]

join_forecast = np.concatenate([
    sc25_main,
    w_cos * sc25_tail + (1 - w_cos) * sc26_head,
    sc26_main
])

lo25_main, lo25_tail = lo25[:-bw], lo25[-bw:]
hi25_main, hi25_tail = hi25[:-bw], hi25[-bw:]
lo26_head, lo26_main = lo26[:bw], lo26[bw:]
hi26_head, hi26_main = hi26[:bw], hi26[bw:]

lo_blend = w_cos * lo25_tail + (1 - w_cos) * lo26_head
hi_blend = w_cos * hi25_tail + (1 - w_cos) * hi26_head

join_lo = np.concatenate([lo25_main, lo_blend, lo26_main])
join_hi = np.concatenate([hi25_main, hi_blend, hi26_main])

print("\nJoined SC25 + SC26 forecast:")
print("  Total length:", len(join_forecast), "months")
print(f"  SC25 peak (offline): {np.max(forecast25):.2f} at {np.argmax(forecast25)}")
print(f"  SC26 peak: {np.max(forecast26):.2f} at {np.argmax(forecast26)}")

fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)
x_join = np.arange(len(join_forecast))
fill_ci(ax, x_join, join_lo, join_hi, label='98% CI', color='0.90')
ax.plot(x_join, join_forecast, label='SC25 + SC26 Forecast', color='black', linewidth=1.8)
ax.set_title("Forecasted Solar Cycles 25 and 26 (Cosine-Blended, 98% CI)")
ax.set_xlabel("Months from SC25 Start (model index)")
ax.set_ylabel("Smoothed SSN")
ax.grid(True, linestyle=':')
ax.legend(fontsize=9, framealpha=1.0)
stamp(ax)
fig.tight_layout()
save_fig(fig, "fig_sc25_sc26_stitched_forecast")


# ============================================
# 8. Observed SC25 vs Offline SC25 forecast
# ============================================

sc24_end = pd.to_datetime(cycle_boundaries[24][1])  # 2019-12-01
sc25_start = sc24_end
sc25_obs = df.loc[sc25_start:CUT_OFF, 'SmoothedSSN'].copy()

print("\nObserved SC25 window (frozen):")
print("  Start:", sc25_obs.index.min().date())
print("  End:  ", sc25_obs.index.max().date())
print("  Observed months:", len(sc25_obs))

obs_months = len(sc25_obs)
forecast_slice = forecast25[:obs_months]
lo_slice = lo25[:obs_months]
hi_slice = hi25[:obs_months]

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
x = np.arange(obs_months)
fill_ci(ax, x, lo_slice, hi_slice, label='98% CI', color='0.90')
ax.plot(x, forecast_slice, label='SC25 Forecast (offline)', color='black', linewidth=1.8)
ax.plot(x, sc25_obs.values, label='Observed SC25 (SILSO)', color='red', linewidth=1.5)
ax.set_title("Observed SC25 vs Offline SC25 Forecast (Data to Nov 2024)")
ax.set_xlabel("Months from SC25 Start (relative index)")
ax.set_ylabel("Smoothed Sunspot Number")
ax.grid(True, linestyle=':')
ax.legend(framealpha=1.0)
stamp(ax)
fig.tight_layout()
save_fig(fig, "fig_sc25_observed_vs_forecast")

print("\nSaved fig_sc25_observed_vs_forecast.*")


# ============================================
# 9. REAL-TIME SC25 FORECAST (DATA ≤ CUT_OFF)
# ============================================

y_obs = sc25_obs.values.astype(float)
x_obs = np.arange(len(y_obs))

print("\n[Real-time SC25] Observed window:")
print(f"  Start: {sc25_obs.index.min().date()}")
print(f"  End:   {sc25_obs.index.max().date()}")
print(f"  Months observed: {len(sc25_obs)}")

if len(sc25_obs) < 12:
    raise RuntimeError("Too few observed SC25 months for a meaningful real-time forecast.")

train_odd_rt = [c for c in cycle_fits.keys() if (c % 2 == 1) and (13 <= c <= 23)]
target = y_obs.copy()
tmin, tmax = np.min(target), np.max(target)
t_norm = (target - tmin) / (tmax - tmin + 1e-9)
t_len = len(target)

distances_rt = []
candidate_curves_rt = {}
for c in train_odd_rt:
    _, y = cycle_fits[c]
    y = np.asarray(y, float)
    ymin, ymax = np.min(y), np.max(y)
    y_norm = (y - ymin) / (ymax - ymin + 1e-9)
    y_adj = np.pad(y_norm, (0, max(0, t_len - len(y_norm))))[:t_len]
    candidate_curves_rt[c] = y_adj
    d, _ = fastdtw(t_norm, y_adj)
    distances_rt.append((c, d))

if not distances_rt:
    raise RuntimeError("No valid odd training cycles found for real-time SC25.")

distances_rt = sorted(distances_rt, key=lambda x: x[1])
k_rt = 2
analog_cycles_rt = [c for c, _ in distances_rt[:k_rt]]

print("\n[Real-time SC25] Selected analog cycles (by DTW distance):", analog_cycles_rt)

analog_lengths = [len(cycle_fits[c][1]) for c in analog_cycles_rt]
L_full = int(round(np.mean(analog_lengths)))
L_full = max(L_full, len(y_obs))

analog_full_norm = []
weights_rt = []
for c, d in distances_rt[:k_rt]:
    _, y = cycle_fits[c]
    y = np.asarray(y, float)
    ymin, ymax = np.min(y), np.max(y)
    y_norm = (y - ymin) / (ymax - ymin + 1e-9)
    y_resampled = np.interp(
        np.linspace(0, 1, L_full),
        np.linspace(0, 1, len(y_norm)),
        y_norm
    )
    analog_full_norm.append(y_resampled)
    weights_rt.append(1.0 / (d + 1e-9))

weights_rt = np.asarray(weights_rt, float)
weights_rt /= weights_rt.sum()
analog_full_norm = np.vstack(analog_full_norm)

blended_norm_rt = weights_rt @ analog_full_norm
analog_amps_rt = [np.max(cycle_fits[c][1]) - np.min(cycle_fits[c][1]) for c in analog_cycles_rt]
analog_bases_rt = [np.min(cycle_fits[c][1]) for c in analog_cycles_rt]
avg_amp_rt = float(np.mean(analog_amps_rt))
avg_base_rt = float(np.mean(analog_bases_rt))

forecast_rt_full = blended_norm_rt * avg_amp_rt + avg_base_rt

join_idx = len(y_obs) - 1
if join_idx < len(forecast_rt_full):
    shift = y_obs[join_idx] - forecast_rt_full[join_idx]
    forecast_rt_full = forecast_rt_full.copy()
    forecast_rt_full[join_idx:] += shift
    forecast_rt_full[join_idx] = y_obs[join_idx]

print(f"\n[Real-time SC25] Forecast length (months): {len(forecast_rt_full)}")
print(f"[Real-time SC25] Peak SSN: {np.max(forecast_rt_full):.2f} at month {np.argmax(forecast_rt_full)}")

lo_rt_full, hi_rt_full = bootstrap_ci_phase_aligned(
    forecast_rt_full,
    odd_bins_zero,
    draws=5000,
    q=(0.01, 0.99),
    seed=2025
)

f_rt_on_obs = forecast_rt_full[:len(y_obs)]
lo_rt_obs = lo_rt_full[:len(y_obs)]
hi_rt_obs = hi_rt_full[:len(y_obs)]

mae_rt = mean_absolute_error(y_obs, f_rt_on_obs)
rmse_rt = true_rmse(y_obs, f_rt_on_obs)
r2_rt = r2_score(y_obs, f_rt_on_obs)
pearson_rt = pearsonr(y_obs, f_rt_on_obs)[0] if (np.std(y_obs) > 0 and np.std(f_rt_on_obs) > 0) else np.nan
mape_rt = mean_absolute_percentage_error(y_obs, f_rt_on_obs)
smape_rt = symmetric_mean_absolute_percentage_error(y_obs, f_rt_on_obs)
within_rt = (y_obs >= lo_rt_obs) & (y_obs <= hi_rt_obs)

print("\n[Real-time SC25] Error metrics over observed window:")
print(f"  MAE   = {mae_rt:.2f}")
print(f"  RMSE  = {rmse_rt:.2f}")
print(f"  R²    = {r2_rt:.2f}")
print(f"  r     = {('nan' if np.isnan(pearson_rt) else f'{pearson_rt:.2f}')}")
print(f"  MAPE  = {mape_rt:.2f}%")
print(f"  SMAPE = {smape_rt:.2f}%")
print(f"  98% CI coverage = {100*np.mean(within_rt):.2f}% ({within_rt.sum()}/{len(within_rt)})")

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
x_full = np.arange(len(forecast_rt_full))
fill_ci(ax, x_full, lo_rt_full, hi_rt_full, label='98% CI (real-time)', color='0.90')
ax.plot(x_full, forecast_rt_full, '--', color='black',
        linewidth=1.8, label='Real-time SC25 forecast (DTW analogs)')
ax.plot(x_obs, y_obs, color='red', linewidth=1.5, label='Observed SC25 (SILSO)')
ax.axvline(x=len(y_obs) - 1, color='0.4', linestyle=':', label='End of observed data')
ax.set_title(f"Real-time SC25 Forecast vs Observations (data ≤ {CUT_OFF.date()})")
ax.set_xlabel(f"Months from SC25 Start ({sc25_start.strftime('%Y-%m')})")
ax.set_ylabel("Smoothed Sunspot Number")
ax.grid(True, linestyle=':')
ax.legend(fontsize=9, framealpha=1.0)
stamp(ax)
fig.tight_layout()
save_fig(fig, "fig_sc25_realtime_vs_observed")

print("Saved fig_sc25_realtime_vs_observed.*")

# Original vs Real-time forecast + Observed
max_len = max(len(forecast25), len(forecast_rt_full))
idx = np.arange(max_len)

def _pad(a, L):
    a = np.asarray(a, float)
    return np.pad(a, (0, L - len(a)), constant_values=np.nan)

f_orig = _pad(forecast25, max_len)
lo_orig = _pad(lo25, max_len)
hi_orig = _pad(hi25, max_len)
f_rt_p = _pad(forecast_rt_full, max_len)
lo_rt_p = _pad(lo_rt_full, max_len)
hi_rt_p = _pad(hi_rt_full, max_len)

fig2 = plt.figure(figsize=(10, 5))
ax2 = fig2.add_subplot(111)
fill_ci(ax2, idx, lo_orig, hi_orig, label='98% CI (offline)', color='0.92')
ax2.plot(idx, f_orig, color='blue', linewidth=1.6, label='Offline SC25 forecast')
fill_ci(ax2, idx, lo_rt_p, hi_rt_p, label='98% CI (real-time)', color='0.85')
ax2.plot(idx, f_rt_p, '--', color='black', linewidth=1.8, label='Real-time SC25 forecast')
ax2.plot(x_obs, y_obs, color='red', linewidth=1.5, label='Observed SC25 (SILSO)')
ax2.axvline(x=len(y_obs) - 1, color='0.4', linestyle=':', label='End of observed data')
ax2.set_title("SC25 Forecasts: Offline vs Real-time vs Observations")
ax2.set_xlabel(f"Months from SC25 Start ({sc25_start.strftime('%Y-%m')})")
ax2.set_ylabel("Smoothed Sunspot Number")
ax2.grid(True, linestyle=':')
ax2.legend(fontsize=8, framealpha=1.0)
stamp(ax2)
fig2.tight_layout()
save_fig(fig2, "fig_sc25_original_vs_realtime")

print("Saved fig_sc25_original_vs_realtime.*")


# ============================================
# 10. Time- and amplitude-normalized shapes (SC13–24) in grid form
# ============================================

def time_and_amp_normalized_cycles(cycle_fits_dict, cycles, n_points=100):
    tnew = np.linspace(0, 1, n_points)
    time_norm = {}
    amp_norm = {}
    for sc in cycles:
        if sc not in cycle_fits_dict:
            continue
        _, y = cycle_fits_dict[sc]
        y = np.asarray(y, float)
        if len(y) < 10:
            continue
        t = np.linspace(0, 1, len(y))
        y_tnorm = np.interp(tnew, t, y)
        ymin, ymax = y_tnorm.min(), y_tnorm.max()
        if ymax > ymin:
            y_ampnorm = (y_tnorm - ymin) / (ymax - ymin)
        else:
            y_ampnorm = np.zeros_like(y_tnorm)
        time_norm[sc] = y_tnorm
        amp_norm[sc] = y_ampnorm
    return tnew, time_norm, amp_norm

cycles_model = list(range(13, 25))
tnew, time_norm_dict, amp_norm_dict = time_and_amp_normalized_cycles(cycle_fits, cycles_model)

fig, axes = plt.subplots(4, 3, figsize=(12, 8), sharex=True, sharey=True)
axes = axes.flatten()
for ax, sc in zip(axes, cycles_model):
    if sc not in time_norm_dict:
        ax.set_visible(False)
        continue
    ax.plot(tnew, time_norm_dict[sc], color='black', linewidth=1.0)
    ax.set_title(f"SC{sc}", fontsize=9)
    ax.grid(True, linestyle=':')
for ax in axes[9:]:
    ax.set_xlabel("Normalized cycle time", fontsize=9)
for ax in axes[::3]:
    ax.set_ylabel("Smoothed SSN", fontsize=9)
fig.suptitle("Time-Normalized Solar Cycle Shapes (SC13–SC24)", y=0.995, fontsize=12)
fig.tight_layout(rect=[0, 0, 1, 0.97])
save_fig(fig, "fig_time_normalized_cycles")

fig, axes = plt.subplots(4, 3, figsize=(12, 8), sharex=True, sharey=True)
axes = axes.flatten()
for ax, sc in zip(axes, cycles_model):
    if sc not in amp_norm_dict:
        ax.set_visible(False)
        continue
    ax.plot(tnew, amp_norm_dict[sc], color='darkgreen', linewidth=1.0)
    ax.set_title(f"SC{sc}", fontsize=9)
    ax.grid(True, linestyle=':')
for ax in axes[9:]:
    ax.set_xlabel("Normalized cycle time", fontsize=9)
for ax in axes[::3]:
    ax.set_ylabel("Normalized amplitude", fontsize=9)
fig.suptitle("Time + Amplitude-Normalized Solar Cycle Shapes (SC13–SC24)", y=0.995, fontsize=12)
fig.tight_layout(rect=[0, 0, 1, 0.97])
save_fig(fig, "fig_amp_normalized_cycles")

# Full smoothed series with horizontal cycle labels
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)
ax.plot(df.index, df['SmoothedSSN'], color='black', linewidth=1)
ymin, ymax = df['SmoothedSSN'].min(), df['SmoothedSSN'].max()
ax.set_ylim(ymin, ymax * 1.10)
for sc, (start_str, end_str) in cycle_boundaries.items():
    start = pd.to_datetime(start_str)
    end = pd.to_datetime(end_str)
    ax.axvline(start, color='0.7', linestyle=':', linewidth=0.8)
    mid = start + (end - start) / 2
    ax.text(
        mid,
        ymax * 1.02,
        f"SC{sc}",
        ha='center',
        va='bottom',
        fontsize=8
    )
ax.set_title("Monthly Smoothed Sunspot Number (SILSO, Frozen at Nov 2024)")
ax.set_xlabel("Year")
ax.set_ylabel("Smoothed SSN")
ax.grid(True, linestyle=':')
stamp(ax)
fig.tight_layout()
save_fig(fig, "fig_full_sunspot_series_label")


# ============================================
# 11. Peak summaries (98% and 80% CI) and SC25 DTW distances
# ============================================

# 98% CI at SC25 offline peak
peak_idx_25 = int(np.argmax(forecast25))
peak_25 = float(forecast25[peak_idx_25])
lo_peak_25 = float(lo25[peak_idx_25])
hi_peak_25 = float(hi25[peak_idx_25])
half_width_25 = (hi_peak_25 - lo_peak_25) / 2.0

print("\n[SC25 offline peak summary]")
print(f"  Peak SSN (point): {peak_25:.2f}")
print(f"  98% CI at peak: [{lo_peak_25:.2f}, {hi_peak_25:.2f}]")
print(f"  Half-width (±): ±{half_width_25:.2f}")

peak_idx_rt = int(np.argmax(forecast_rt_full))
peak_rt = float(forecast_rt_full[peak_idx_rt])
lo_peak_rt = float(lo_rt_full[peak_idx_rt])
hi_peak_rt = float(hi_rt_full[peak_idx_rt])
half_width_rt = (hi_peak_rt - lo_peak_rt) / 2.0

print("\n[SC25 real-time peak summary]")
print(f"  Peak SSN (point): {peak_rt:.2f}")
print(f"  98% CI at peak: [{lo_peak_rt:.2f}, {hi_peak_rt:.2f}]")
print(f"  Half-width (±): ±{half_width_rt:.2f}")

# 80% CI bands
lo25_80, hi25_80 = bootstrap_ci_phase_aligned(
    forecast25,
    odd_bins_zero,
    draws=5000,
    q=(0.10, 0.90),
    seed=RNG_SEED
)

lo_rt_full_80, hi_rt_full_80 = bootstrap_ci_phase_aligned(
    forecast_rt_full,
    odd_bins_zero,
    draws=5000,
    q=(0.10, 0.90),
    seed=2025
)

lo_peak_25_80 = float(lo25_80[peak_idx_25])
hi_peak_25_80 = float(hi25_80[peak_idx_25])
half_width_25_80 = (hi_peak_25_80 - lo_peak_25_80) / 2.0

print("\n[SC25 offline peak summary — 80% CI]")
print(f"  Peak SSN (point): {peak_25:.2f}")
print(f"  80% CI at peak: [{lo_peak_25_80:.2f}, {hi_peak_25_80:.2f}]")
print(f"  Half-width (±): ±{half_width_25_80:.2f}")

lo_peak_rt_80 = float(lo_rt_full_80[peak_idx_rt])
hi_peak_rt_80 = float(hi_rt_full_80[peak_idx_rt])
half_width_rt_80 = (hi_peak_rt_80 - lo_peak_rt_80) / 2.0

print("\n[SC25 real-time peak summary — 80% CI]")
print(f"  Peak SSN (point): {peak_rt:.2f}")
print(f"  80% CI at peak: [{lo_peak_rt_80:.2f}, {hi_peak_rt_80:.2f}]")
print(f"  Half-width (±): ±{half_width_rt_80:.2f}")

# SC25 DTW distances (odd cycles, for analog justification)
odd_pool_all_sc25 = [c for c in range(13, 24) if (c % 2 == 1) and (c in cycle_fits)]
ref25 = generate_convex_reference(sc25_length)
distances_odd = []
for c in odd_pool_all_sc25:
    _, y = cycle_fits[c]
    y = np.asarray(y, float)
    y_norm = (y - y.min()) / (y.max() - y.min() + 1e-9)
    y_adj = np.pad(y_norm, (0, max(0, len(ref25) - len(y_norm))))[:len(ref25)]
    r25_norm = (ref25 - ref25.min()) / (ref25.max() - ref25.min() + 1e-9)
    d, _ = fastdtw(r25_norm, y_adj)
    distances_odd.append((c, d))

distances_odd = sorted(distances_odd, key=lambda x: x[1])

print("\nDTW distances to SC25 reference (odd cycles):")
for c, d in distances_odd:
    print(f"  SC{c}: {d:.2f}")

fig = plt.figure(figsize=(8, 5))
ax = fig.add_subplot(111)
labels = [f"SC{c}" for (c, _) in distances_odd]
vals = [d for (_, d) in distances_odd]
bars = ax.bar(labels, vals, color='0.8')
selected = [19, 13]
for bar, (c, _) in zip(bars, distances_odd):
    if c in selected:
        bar.set_color('black')
ax.set_ylabel("DTW Distance")
ax.set_title("DTW Distance of Odd Cycles to SC25 Reference Curve")
ax.grid(True, axis='y', linestyle=':')
fig.tight_layout()
save_fig(fig, "fig_sc25_dtw_distances")

print("\nPipeline finished.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Frozen SILSO range: 1749-07-01 → 2024-11-01 | rows: 3305
Cycles loaded: [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Saved fig_full_sunspot_series.pdf and fig_full_sunspot_series.eps

=== Validation Summary (SC20–24) ===
k      Avg MAE  Avg RMSE    Avg R2     Avg r    Avg MAPE   Avg SMAPE
--------------------------------------------------------------------
2        11.94     15.85      0.92      0.97       23.62       23.07
3        13.33     16.97      0.92      0.97       27.90       23.99
4        13.53     17.02      0.92      0.97       27.30       24.06
--------------------------------------------------------------------

Best k based on average RMSE: k=2
Saved fig_k_comparison_sc24.pdf and fig_k_comparison_sc24.eps
Saved k-comparison figure for SC24 as fig_k_comparison_sc24.pdf/.eps
Saved fig_sc20_24_validation_grid.pdf and fig_sc20_24_validation_grid.eps